***Python*** ***Assignment***

In [7]:

!pip install -q dspy-ai>=2.5.0 anthropic pydantic beautifulsoup4 requests pandas

import dspy
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pydantic import BaseModel, Field
from typing import List
import re
import time
import os
from google.colab import userdata

print("✅ Environment Ready.")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires grpcio>=1.71.2, but you have grpcio 1.67.1 which is incompatible.
✅ Environment Ready.


In [8]:
# @title 2. Secure API Configuration
CONFIRMED_MODEL = "anthropic/claude-sonnet-4-20250514"

try:
    api_key = userdata.get('ANTHROPIC_API_KEY')
    # api_base = userdata.get('LONGCAT_BASE_URL') # Uncomment if using proxy
    api_base = None

    lm = dspy.LM(
        model=CONFIRMED_MODEL,
        api_key=api_key,
        api_base=api_base,
        temperature=0.0
    )
    dspy.configure(lm=lm)
    print(f"✅ Connected to: {CONFIRMED_MODEL}")

except userdata.SecretNotFoundError:
    print("❌ Error: 'ANTHROPIC_API_KEY' is missing from Secrets.")
except Exception as e:
    print(f"❌ Connection Error: {e}")

✅ Connected to: anthropic/claude-sonnet-4-20250514


In [9]:
# @title 3. Define Pydantic Models & DSPy Signatures

# 1. Entity Extraction
class EntityItem(BaseModel):
    name: str = Field(description="Exact entity name (e.g. 'pea-barley intercrop').")
    category: str = Field(description="Semantic category (e.g. Crop, Process, Concept).")

class EntityList(BaseModel):
    entities: List[EntityItem]

class ExtractEntitiesSig(dspy.Signature):
    """Extract key domain entities from text. Return structured JSON."""
    text: str = dspy.InputField()
    output: EntityList = dspy.OutputField()

# 2. Deduplication
class DeduplicationOutput(BaseModel):
    deduplicated_entities: List[EntityItem]
    confidence_score: float = Field(description="Confidence score (0.0 - 1.0).")

class DeduplicateSig(dspy.Signature):
    """Merge synonyms and remove duplicates. Provide confidence score."""
    raw_entities: List[EntityItem] = dspy.InputField()
    output: DeduplicationOutput = dspy.OutputField()

# 3. Knowledge Graph
class Relationship(BaseModel):
    source: str = Field(description="Source node (must be from valid_entities).")
    target: str = Field(description="Target node (must be from valid_entities).")
    label: str = Field(description="Edge label (max 40 chars).")

class GraphOutput(BaseModel):
    relationships: List[Relationship]

class BuildGraphSig(dspy.Signature):
    """Identify relationships using ONLY the provided valid entities."""
    text: str = dspy.InputField()
    valid_entities: List[str] = dspy.InputField()
    output: GraphOutput = dspy.OutputField()

# Initialize Predictors
extractor = dspy.Predict(ExtractEntitiesSig)
deduplicator = dspy.Predict(DeduplicateSig)
graph_builder = dspy.Predict(BuildGraphSig)

print("✅ DSPy Modules Initialized.")

✅ DSPy Modules Initialized.


In [10]:
# @title 4. Define Utilities

def scrape_and_clean(url: str) -> str:
    """Robust scraper with noise removal."""
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0 Safari/537.36"}
    try:
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Kill noise
        for el in soup(['script', 'style', 'nav', 'footer', 'header', 'form', 'iframe', 'aside']):
            el.decompose()

        text = re.sub(r'\s+', ' ', soup.get_text(separator=' ')).strip()

        return text[:20000]
    except Exception as e:
        print(f"⚠️ Scrape Error for {url}: {e}")
        return ""

def generate_mermaid(rels) -> str:
    """Convert relationships to Mermaid Syntax."""
    lines = ["graph TD"]
    for r in rels:
        s = r.source.replace('"', '').replace('(', '').replace(')', '')
        t = r.target.replace('"', '').replace('(', '').replace(')', '')
        l = r.label.replace('"', '')[:40]
        lines.append(f'    "{s}" -- "{l}" --> "{t}"')
    return "\n".join(lines)

def robust_deduplicate_logic(entities):
    """Wrapper for deduplication predictor."""
    if not entities: return []
    try:
        res = deduplicator(raw_entities=entities).output
        print(f"      > Dedup Confidence: {res.confidence_score}")
        return res.deduplicated_entities
    except:
        return entities

print("✅ Utilities Ready.")

✅ Utilities Ready.


In [11]:
# @title 5. Execute Full Pipeline
urls = [
    "https://en.wikipedia.org/wiki/Sustainable_agriculture",
    "https://www.nature.com/articles/d41586-025-03353-5",
    "https://www.sciencedirect.com/science/article/pii/S1043661820315152",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10457221/",
    "https://www.fao.org/3/y4671e/y4671e06.htm",
    "https://www.medscape.com/viewarticle/time-reconsider-tramadol-chronic-pain-2025a1000ria",
    "https://www.sciencedirect.com/science/article/pii/S0378378220307088",
    "https://www.frontiersin.org/news/2025/09/01/rectangle-telescope-finding-habitable-planets",
    "https://www.medscape.com/viewarticle/second-dose-boosts-shingles-protection-adults-aged-65-years-2025a1000ro7",
    "https://www.theguardian.com/global-development/2025/oct/13/astro-ambassadors-stargazers-himalayas-hanle-ladakh-india"
]

results = []
print("🚀 Starting Batch Processing...")
print("="*60)

for i, url in enumerate(urls):
    print(f"\n🔗 Processing {i+1}/10: {url}")

    # 1. Scrape
    text = scrape_and_clean(url)
    if not text:
        print("      > ⚠️ Skipping (Scrape failed)")
        continue

    # 2. Extract
    try:
        raw_ents = extractor(text=text).output.entities
        print(f"      > Extracted {len(raw_ents)} raw entities")
    except Exception as e:
        print(f"      > Extraction Failed: {e}")
        continue

    # 3. Deduplicate
    clean_ents = robust_deduplicate_logic(raw_ents)
    print(f"      > Cleaned to {len(clean_ents)} unique entities")

    # 4. Graph Generation
    try:
        valid_names = [e.name for e in clean_ents]
        rels = graph_builder(text=text[:15000], valid_entities=valid_names).output.relationships
        mermaid_code = generate_mermaid(rels)

        # Save Mermaid File
        fname = f"mermaid_{i+1}.md"
        with open(fname, "w") as f:
            f.write(mermaid_code)
        print(f"      > Saved {fname}")
    except Exception as e:
        print(f"      > Graph Generation Failed: {e}")

    # 5. Collect CSV Data
    for e in clean_ents:
        results.append({"link": url, "tag": e.name, "tag_type": e.category})

    time.sleep(1)

# 6. Export CSV
print("="*60)
if results:
    df = pd.DataFrame(results)
    df.drop_duplicates(subset=['link', 'tag'], inplace=True)
    df.to_csv("tags.csv", index=False)
    print(f"🎉 SUCCESS! Generated 'tags.csv' ({len(df)} rows) and 10 Mermaid files.")
else:
    print("⚠️ No data collected.")

🚀 Starting Batch Processing...

🔗 Processing 1/10: https://en.wikipedia.org/wiki/Sustainable_agriculture
      > Extracted 51 raw entities
      > Dedup Confidence: 0.95
      > Cleaned to 49 unique entities
      > Saved mermaid_1.md

🔗 Processing 2/10: https://www.nature.com/articles/d41586-025-03353-5
      > Extracted 15 raw entities
      > Dedup Confidence: 0.95
      > Cleaned to 15 unique entities
      > Saved mermaid_2.md

🔗 Processing 3/10: https://www.sciencedirect.com/science/article/pii/S1043661820315152
⚠️ Scrape Error for https://www.sciencedirect.com/science/article/pii/S1043661820315152: 403 Client Error: Forbidden for url: https://www.sciencedirect.com/science/article/pii/S1043661820315152
      > ⚠️ Skipping (Scrape failed)

🔗 Processing 4/10: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10457221/
⚠️ Scrape Error for https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10457221/: 403 Client Error: Forbidden for url: https://pmc.ncbi.nlm.nih.gov/articles/PMC10457221/
      >

In [12]:
# @title 6. Preview Output
import os

print("### File Check ###")
files = os.listdir('.')
md_files = [f for f in files if f.startswith('mermaid_') and f.endswith('.md')]
print(f"Mermaid Files Found: {len(md_files)}")
print(f"CSV Found: {'tags.csv' in files}")

if 'tags.csv' in files:
    print("\n### CSV Preview ###")
    print(pd.read_csv("tags.csv").head())

if md_files:
    print(f"\n### {md_files[0]} Preview ###")
    with open(md_files[0], 'r') as f:
        print(f.read())

### File Check ###
Mermaid Files Found: 5
CSV Found: True

### CSV Preview ###
                                                link                      tag  \
0  https://en.wikipedia.org/wiki/Sustainable_agri...  sustainable agriculture   
1  https://en.wikipedia.org/wiki/Sustainable_agri...       shade-grown coffee   
2  https://en.wikipedia.org/wiki/Sustainable_agri...              polyculture   
3  https://en.wikipedia.org/wiki/Sustainable_agri...       ecosystem services   
4  https://en.wikipedia.org/wiki/Sustainable_agri...           climate change   

               tag_type  
0               Concept  
1                  Crop  
2               Process  
3               Concept  
4  Environmental Factor  

### mermaid_8.md Preview ###
graph TD
    "rectangular telescope" -- "could help discover" --> "Earth 2.0"
    "rectangular telescope" -- "modeled after" --> "Diffractive Interfero Coronagraph Exoplanet Resolver"
    "rectangular telescope" -- "similar size to" --> "James Webb